In [1]:
# External libraries
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [2]:
# LOAD ALL IMAGES:
dt = 10
dict = {}
for img in glob.glob(f"DATA/ephemeris sat/inclination zero/{dt} step size/*"):
    txt = img             # stringa
    t = txt.split('_')[1] # numero
    dict[t] = txt

In [4]:
# df = pd.read_csv('DATA/H_L_combined.csv')
DB = pd.read_csv('DATA/lunar_crater_database_robbins_2018.csv')
craters_cat = CatalogSearch(DB, lat_bounds=[-1.5, 1.5], lon_bounds=[-129.802-1.5,-129.802+1.5], CAT_NAME='ROBBINS')

deg2km = 2*np.pi*1737.4/360
km2deg = 1/deg2km
craters_cat = craters_cat[(craters_cat.Diam < 35)&(craters_cat.Diam > 3)]
craters_cat['Diam']*=0.5*km2deg
craters_cat.rename( columns={'Diam':'r_deg'}, inplace=True)

craters_cat_m = np.array(craters_cat)
craters_cat_m.shape
t1 = time.time()
triplets_cat_m = find_all_triplets(craters_cat_m)
triplets_cat = pd.DataFrame(triplets_cat_m, columns=['Angle1','Angle2','Angle3','des1','des2','des3','lon1','lat1','r1','lon2','lat2','r2','lon3','lat3','r3'])
triplets_cat.shape
t2 = time.time()
print(t2-t1)

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
0.012276887893676758


In [5]:
idx = 34
# IMG1:
filename = dict[str(idx+1)]
img=cv2.imread(filename)
craters_det = detect(img)

# Pandas DataFrame:
df_craters_det = sort_mat(craters_det)
# Find all triplets:
t1 = time.time()
triplets = find_all_triplets(craters_det)
triplets_det= pd.DataFrame(triplets, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
triplets_det.shape
t2 = time.time()
print(t2-t1)

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
0.06864690780639648


In [77]:
t1 = time.time()
QUERY1 = triplets_cat
QUERY2 = triplets_det 
QUERY1 =QUERY1[ (QUERY1.Angle1 > 10) & (QUERY1.Angle2 > 10) & (QUERY1.Angle3 > 10) ].reset_index(drop=True)
QUERY2 =QUERY2[ (QUERY2.Angle1 > 10) & (QUERY2.Angle2 > 10) & (QUERY2.Angle3 > 10) ].reset_index(drop=True)
joins, items = inner_join(QUERY1, QUERY2, 15)

t2 = time.time()
deltat = t2-t1
print(deltat)

0.8449349403381348


In [78]:
len(joins)

0

In [66]:
def inner_join(q1, q2, tol1):
    DFs, items = [], []
    s = pd.DataFrame()
    for ind, row in q1.iterrows():

        cond1 = ((abs(row.Angle1 - q2.Angle1)< tol1))
        cond2 = ((abs(row.Angle1 - q2.Angle2)< tol1))
        cond3 = ((abs(row.Angle1 - q2.Angle3)< tol1))
        
        if np.any(cond1):
            cond1_a = ((abs(row.Angle2 - q2.Angle2)< tol1))
            cond1_b = ((abs(row.Angle3 - q2.Angle3)< tol1))
            cond1_c = ((abs(row.Angle2 - q2.Angle3)< tol1))
            cond1_d = ((abs(row.Angle3 - q2.Angle2)< tol1))

            cond1_A = (np.any(cond1_a) & np.any(cond1_b))
            cond1_B = (np.any(cond1_c) & np.any(cond1_d))

            if cond1_A & (not cond1_B):
                s = q2[cond1 & cond1_a & cond1_b]
            elif cond1_B & (not cond1_A):
                s = q2[cond1 & cond1_c & cond1_d]
        
        elif np.any(cond2):
            cond2_a = ((abs(row.Angle2 - q2.Angle1)< tol1))
            cond2_b = ((abs(row.Angle3 - q2.Angle3)< tol1))
            cond2_c = ((abs(row.Angle2 - q2.Angle3)< tol1))
            cond2_d = ((abs(row.Angle3 - q2.Angle1)< tol1))

            cond2_A = (np.any(cond2_a) & np.any(cond2_b))
            cond2_B = (np.any(cond2_c) & np.any(cond2_d))

            if cond2_A & (not cond2_B):
                s = q2[cond2 & cond2_a & cond2_b]
            elif cond1_B & (not cond2_A):
                s = q2[cond2 & cond2_c & cond2_d]

        elif np.any(cond3):
            cond3_a = ((abs(row.Angle2 - q2.Angle1)< tol1))
            cond3_b = ((abs(row.Angle3 - q2.Angle2)< tol1))
            cond3_c = ((abs(row.Angle2 - q2.Angle2)< tol1))
            cond3_d = ((abs(row.Angle3 - q2.Angle1)< tol1))

            cond3_A = (np.any(cond2_a) & np.any(cond2_b))
            cond3_B = (np.any(cond2_c) & np.any(cond2_d))

            if cond3_A & (not cond3_B):
                s = q2[cond3 & cond3_a & cond3_b]
            elif cond1_B & (not cond2_A):
                s = q2[cond3 & cond3_c & cond3_d]


        if s.shape[0] !=0:
            DFs.append(s)
            items.append(row)

    return DFs, items

In [10]:
if len(joins)>1:
    print(len(joins))
    plt.close()
    i = 6
    arg = items[i][0]
    a = QUERY1.iloc[arg]
    crts1 = np.vstack([[a.x1,a.y1,a.r1],[a.x2,a.y2,a.r2],[a.x3,a.y3,a.r3]])
    
    plt.figure(dpi=200)
    plt.subplot(121)
    cp1 = img.copy()
    IMG1 =  img_plus_crts(cp1, crts1, color="red")
    plt.imshow(IMG1)
    plt.show()

    join = joins[i]
    j=1
    b = join.iloc[j]
    crts2 = np.vstack([[b.lon1,b.lat1,b.r1],[b.lon2,b.lat2,b.r2],[b.lon3,b.lat3,b.r3]])
    crts2[:,2]*= deg2km
    plt.subplot(122)
    lat_bounds=[-1.5, 1.5]
    lon_bounds=[-129.802-1.5,-129.802+1.5]
    crts2_to_rel = craters_to_relative_frame(crts2, lon_b=lon_bounds, lat_b=lat_bounds)
    IMG2 =  img_plus_crts(cp1, crts2_to_rel, color="green")
    plt.imshow(IMG2)
    plt.show()

In [ ]:
c2 = compute_centroid(crts2)
c1 = compute_centroid(crts1)

In [ ]:
# FIG.2
cp1 = deepcopy(img)
DB = pd.read_csv('DATA/lunar_crater_database_robbins_2018.csv')
df = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME='ROBBINS')
df = df[df.Diam < 35]
image_with_craters = draw_craters_on_image(df,  lon_bounds, lat_bounds, cp1, u=None)

plt.ion()
plt.figure('Image with craters catalogued', dpi=200)
plt.imshow(image_with_craters)
plt.show()

In [ ]:
cp1 = deepcopy(img)
img_det = img_plus_crts(img, craters_det)
plt.figure()
plt.imshow(img_det)
plt.show()

In [ ]:
def craters_to_relative_frame(df, lon_b, lat_b, u=None):
    lon_bounds = lon_b
    lat_bounds = lat_b
    # CAMERA CENTER:
    CAMx, CAMy = (
        (lon_bounds[0] + lon_bounds[1]) / 2,
        (lat_bounds[0] + lat_bounds[1]) / 2,
    )

    if u == None:  # Scale Factor
        u = 257.52  # ? DEG TO PXS
        span = (abs(lon_b[0]) - abs(lon_b[1])) * u
        span = abs(int(span))
        print(span)
    else:
        span = (abs(lon_b[0]) - abs(lon_b[1])) * u
        span = abs(int(span))
    # Make the img:
    # img = np.zeros((span, span), dtype=int)
    if df is None:
        print("No crater found")
        pass
    else:
        W, H = (span, span)
        # Cycle through the dataframe:
        craters = np.zeros(3)
        for i in range(df.shape[0]):
            crater = df[i]
            # crater center:
            xc, yc = crater[0], crater[1]  # This is in the absolute frame
            # f: Absolute --> f: Relative
            xc = xc - CAMx
            yc = yc - CAMy
            # f: relative --> f: OPENCV
            xc *= u  # Now is in pixel not in lon deg
            yc *= u  # Now is in pixel not in lat deg
            xc = W / 2 + xc
            yc = H / 2 - yc
            # ? 1 km = 8.4746 px in our DEM := Merge LOLA - KAGUYA
            KM_to_PX = 1/0.118
            crater_i = [xc, yc, crater[2] * KM_to_PX]
            craters = np.vstack([craters, crater_i])
        return craters[1:, :]